In [1]:
import time

import numpy as np
import tensorflow as tf

In [2]:
with tf.Session() as sess:
  new_saver = tf.train.import_meta_graph('checkpoints/text8.ckpt.meta')
  new_saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))

INFO:tensorflow:Restoring parameters from checkpoints/text8.ckpt


In [3]:
with open('albaniantext') as f:
    text = f.read()

In [4]:
from collections import Counter

In [5]:
words = text.split()
    
# Remove all words with  5 or fewer occurences
word_counts = Counter(words)
trimmed_words = [word for word in words if word_counts[word] > 5]

In [6]:
def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict....
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

In [7]:
vocab_to_int, int_to_vocab = create_lookup_tables(trimmed_words)
int_words = [vocab_to_int[word] for word in trimmed_words]

In [8]:
from collections import Counter
import random

threshold = 1e-5
word_counts = Counter(int_words)
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
print(max(p_drop.values()))
print(min(p_drop.values()))
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

0.9863050084018772
-5.2627752101870415


In [9]:

def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    
    return list(target_words)

In [10]:

def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [11]:
inputs = tf.placeholder(tf.int32, [None], name='inputs')
labels = tf.placeholder(tf.int32, [None, None], name='labels')
n_vocab = len(int_to_vocab)
n_embedding = 200 # Number of embedding features 
embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
embed = tf.nn.embedding_lookup(embedding, inputs)

In [12]:
n_sampled = 100
softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
softmax_b = tf.Variable(tf.zeros(n_vocab))

# Calculate the loss using negative sampling
loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, 
                                  labels, embed,
                                  n_sampled, n_vocab)

cost = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer().minimize(cost)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [13]:
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100
# pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
valid_examples = np.append(valid_examples, 
                           random.sample(range(1000,1000+valid_window), valid_size//2))

valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

# We use the cosine distance:
norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
normalized_embedding = embedding / norm
valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [18]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    word = 'kosova'
    example = []
    example.append(vocab_to_int[word])
    valid_dataset = tf.constant(example, dtype=tf.int32)

    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))

    normalized_embedding = embedding / norm

    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    print(valid_embedding)
    similarity2 = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))
    sim2 = similarity2.eval()



    valid_word = word
    top_k = 8
    nearest = (-sim2[0, :]).argsort()[1:top_k+1]
    print(nearest)
    log = 'Nearest to %s:' % valid_word
    for k in range(top_k):
        close_word = int_to_vocab[nearest[k]]
        log = '%s %s,' % (log, close_word)
    print(log)

Tensor("embedding_lookup_8/Identity:0", shape=(1, 200), dtype=float32)
[87072 10617 88186 52176 94873 80370 72279 57453]
Nearest to kosova: backgroundcolor, pafund, daher, finch, tàrrega, katigoria, aoki, kasia,
